# Tutorial on inference with SPEXAI

This tutorial explains how you can use SPEXAI to fit observered spectra from a FITS file.

In [ ]:
import numpy as np

from spexai import FitTempDist

## Setup the ensemble sampler

The paramater that are fitted are the mean temperature in KeV  ```temp```, standart deviation on the temperature distribution in log10(KeV) ```stdevtemp```, metalicity [solar] ```met``` , single element ```X``` abundace with respect to Iron ```Z_```, turbulent velocity in km/sec ```vel``` and redshift in log10(z) ```logz```. The intial guess with there standard diviation can be put into the dictornary of the ```prior```.

The ensemble samper can be intialized with ```nwalkers``` and ```nsteps``` for the walker to go through, other parameter that can be initialized are the Luminosity Distance in m and the energy interval of the spectrum in KeV (```e_min, e_max```).

In [ ]:
prior = {'temp': {'mu': 5, 'sigma': 2},
        'stdevtemp': {'mu': -5, 'sigma': 2},
        'met':  {'mu': 1, 'sigma': .3},
        'Z_':{'mu': 1, 'sigma': .3},
        'vel':  {'mu': 100, 'sigma': 50},
        'norm': {'mu': 1e10, 'sigma': 1e10},
        'logz': {'mu': -5,  'sigma': 2}
        }
fit = FitTempDist(50, 200, Luminosity_Distance=9.461E+24, prior=prior, e_min=2,e_max=9)

### Reading in the FITS files

To be able to fit real data the response of the telescope can be read in by the FITS file, for the Response Matrix File (RMF), effective area response file (ARF) and the FITS file of the observed data.

In addition the response files there is a sparse matrix (```make_sparsex```) used for convulation that implements line broadening to the spectra. The speed an accuracy of the line broadening is strongly dependent on the kernel size of the convulolution ```n``` and the default ```n=300```, increasing ```n``` will make the line-broading more accurete but will also segnifcanly impact the speed.

In [ ]:
'''This step can take a long time to run'''
#reading in the response matrix file
fit.combined_model.load_rm('Path_RMF')

In [ ]:
#reading in the effective area
fit.combined_model.load_arf('Path_ARF')

In [ ]:
'''This step can take a long time to run'''
#initializing the sparse matrix for the convolution from line-broadening
fit.combined_model.load_sparsematrix_x(n=300)

In [ ]:
#reading in the data
fit.load_data('Path_data')

#### Add aditional parameters

Single element abundace that differ with respect to the overall metalicity and Iron can be added in as extra parameter(s).
These are writen in the format ```'ZX [ZX/Fe]'``` with ```X``` the atom number of the elements. ```add_position``` gives the intial values of the sampler for the added paramater(s).

In [ ]:
#names of aditional fitted parameters
add_params = ['Z8 [Z8/Fe]','Z14 [Z14/Fe]']
#intial position walkers of the initial parameters
add_position =  np.concatenate((np.random.normal(1, .3, size=(1, fit.nwalkers)),
                                np.random.normal(1, .3, size=(1, fit.nwalkers))), axis=0)

Then after ensemble sampler has been fully intialized the data can be fitted with ```FitTempDist.fit_spectra``` making use of ```emcee``` algorithm.
```FitTempDist.fit_spectra``` will aslo print the the integrated autocorrelation time to give an indication of the burn-in time.

In [ ]:
#fitting the data
np.seterr(all="ignore")
fit.fit_spectra(add_params=add_params, add_position=add_position)

## Evaluating the fit results

The progress of the walker can be visualized in a timeseries giving the parameter values for each walker at each step in the chain.

In [ ]:
fit.plot_timeseries()

The results of the fit can visualized with a cornerplot. ```FitTempDist.cornerplot``` is also able to save the sampled posterior in an .csv format by giving a directory name.  In this example you discard the first ```200``` steps in the chain and only reading every 15th step. The corner plot can also be oveploted with the true values of the parameters by giving a dictionary ```true_values```.

In [ ]:
fit.cornerplot(200, thin=15, filedir=None)

The fit results can be directly compared to the data by overplotting the fitted model with parameters drawn from the posterior to the observed data. Also plots the mean and 1-sigma interval of the fitted parameters.

In [ ]:
fit.plot_spectrum()